# Chapter 7 - Dimensionality Reduction

Paul E. Anderson

In [2]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. change to something else if this is not the case on your system

We saw in the last chapter that it was difficult to visualize more than two dimensions. What are our options? Choose two dimensions to visualize at time? Plot three dimensions that are then projected down into two dimensions? What if you could create new dimensions that are "optimized" combinations of the original dimensions? 

Finding/creating those new dimensions is what this chapter is all about. There are many different ways to create new dimensions, and we will focus on one of the most popular methods: Principal Component Analysis (PCA). 

## Example motivation: Visualizing three gene data

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv(
    f"{home}/csc-466-student/data/breast_cancer_three_gene.csv",index_col=0
)
df.head()

,ESR1,AURKA,ERBB2,Subtype
0,0.804501,0.264356,6.941677,LumA
1,0.163597,0.589052,6.551394,Basal
2,0.569347,0.189531,7.056530,LumA
3,0.847584,0.264849,7.028625,LumB
4,0.442474,0.526040,8.783604,LumB


In [4]:
X = df[['ESR1','AURKA','ERBB2']]
X.head()

,ESR1,AURKA,ERBB2
0,0.804501,0.264356,6.941677
1,0.163597,0.589052,6.551394
2,0.569347,0.189531,7.056530
3,0.847584,0.264849,7.028625
4,0.442474,0.526040,8.783604


In [5]:
# Three ways to plot
import altair as alt

g1 = alt.Chart(X).mark_point().encode(
    x='ESR1',
    y='AURKA'
)

g2 = alt.Chart(X).mark_point().encode(
    x='ESR1',
    y='ERBB2'
)

g3 = alt.Chart(X).mark_point().encode(
    x='AURKA',
    y='ERBB2'
)

In [6]:
g1

alt.Chart(...)

In [7]:
g2

alt.Chart(...)

In [8]:
g3

alt.Chart(...)

It would be very nice to have all of this on a single plot? So we are left with wondering how can we do that in a way that minimizes the amount of information that we are not seeing?

In [9]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
Xnew = pd.DataFrame(pca.fit_transform(X),columns=["PC1","PC2","PC3"])
Xnew.head()

,PC1,PC2,PC3
0,-0.260440,-0.258075,0.014488
1,-0.596121,0.458796,0.219232
2,-0.137859,-0.050679,-0.112194
3,-0.176034,-0.305667,0.019684
4,1.610413,0.030249,0.109422


**Stop and think:** What is this printing?

In [10]:
print(pca.explained_variance_ratio_)

[0.75246059 0.19295907 0.05458034]


**Your answer here**

**Stop and think:** What is this printing?

In [11]:
print(pca.singular_values_)

[24.05038031 12.1790291   6.47736201]


**Your answer here**

**Stop and think:** What is this printing?

In [12]:
print(pca.components_)

[[-0.05325979  0.05921908  0.9968232 ]
 [-0.97739922  0.20142163 -0.06418799]
 [ 0.20458291  0.97771286 -0.04715301]]


**Your answer here**

### We can now visualize our data!

In [13]:
source = Xnew.copy()
source['Subtype'] = df['Subtype']

alt.Chart(source).mark_point().encode(
    x='PC1',
    y='PC2',
    color='Subtype'
)

alt.Chart(...)

## PCA from scratch
To understand how all of this is working, we will implement a PCA that goes from two dimensions to one. It will match sklearn's answers.

We will reduce two dimensions into a single dimension. The work we do here, can easily be extended to more than two dimensions.

In [14]:
X = df[['ESR1','AURKA']] # we will reduce from two variables to one
X.head()

,ESR1,AURKA
0,0.804501,0.264356
1,0.163597,0.589052
2,0.569347,0.189531
3,0.847584,0.264849
4,0.442474,0.526040


### Run PCA and summarize

In [15]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
Xnew = pd.DataFrame(pca.fit_transform(X),columns=["PC1","PC2"])
print('Components')
print(pca.components_)
print('Singular values')
print(pca.singular_values_)
print('Explained variance ratio')
print(pca.explained_variance_ratio_)

Components
[[-0.97623958  0.21669397]
 [ 0.21669397  0.97623958]]
Singular values
[12.25236834  6.56728546]
Explained variance ratio
[0.77682127 0.22317873]


In [16]:
source = Xnew.copy()
source['Subtype'] = df['Subtype']

g = alt.Chart(source).mark_point().encode(
    x='PC1',
    y='PC2',
    color='Subtype'
)

### Visualize

In [17]:
g

alt.Chart(...)

**Stop and think:** What is the covariance matrix?

In [23]:
COV = pd.DataFrame(columns=X.columns,index=X.columns)
COV
# Your solution here

,ESR1,AURKA
ESR1,0.068057,-0.010616
AURKA,-0.010616,0.022586


In [24]:
# Cheat code:
X.cov()

,ESR1,AURKA
ESR1,0.068057,-0.010616
AURKA,-0.010616,0.022586


#### Recall from slides:

$\lambda^2 - (a + d)\lambda + (ad - bc) = 0$

where

In [25]:
a = COV.iloc[0,0]
b = COV.iloc[0,1]
c = COV.iloc[1,0]
d = COV.iloc[1,1]

#### Find those roots!

In [26]:
import numpy as np
# See np.roots
## BEGIN SOLUTUION
coeff = [1,-(a+d),(a*d - b*c)]
lambdas = np.roots(coeff)
# Your solution here
lambdas

array([0.07041301, 0.02022947])

### Recall from slides:

$(\Sigma - I*\lambda)\vec{v} = 0$

If $\vec{v} = (x,y)$, then

$(a-\lambda)*x + b*y = 0$ and $c*x + (d-\lambda)*y = 0$

To solve either of these equations, let $x=1$ and then solve for $y$. Take that vector and normalize it to have a length of 1.

In [27]:
# Your solution here
y

array([-0.22196802,  4.50515352])

### Normalize the vectors to have a length of 1

In [28]:
v1 = np.array([1,y[0]])
v1 = v1/np.sqrt(np.sum(v1**2))
v2 = np.array([1,y[1]])
v2 = v2/np.sqrt(np.sum(v2**2))
print("First eigen vector:",v1)
print("Second eigen vector:",v2)

First eigen vector: [ 0.97623958 -0.21669397]
Second eigen vector: [0.21669397 0.97623958]


**Stop and think:** Are these identical? If not, are they equivalent?

Your solution here

**Stop and think:** How do you project a sample into the new first dimension?

In [30]:
## BEGIN SOLTUION
PC1_score = np.sum(X.iloc[0]*v1)
# Your solution here
PC1_score

0.7281010377879399

#### Cheat code

In [32]:
PC1 = np.dot(X,v1)
PC1

array([ 0.72810104,  0.03206596,  0.51474887, ...,  0.59394431,
       -0.01993961,  0.60444599])

In [33]:
PC2 = np.dot(X,v2)
PC2

array([0.43240559, 0.61050596, 0.30840134, ..., 0.38077829, 0.48725345,
       0.43733798])

In [35]:
source = pd.DataFrame({"PC1":PC1,"PC2":PC2,"Subtype":df["Subtype"]})

g = alt.Chart(source).mark_point().encode(
    x='PC1',
    y='PC2',
    color='Subtype'
)

### Plot the results!

**Stop and think** Does this look the same as above?

Your answer here

In [37]:
g

alt.Chart(...)

## What about as a preprocessing step?

In [48]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
import numpy as np
import sklearn

X = df[['ESR1','AURKA','ERBB2']]
t = pd.get_dummies(df['Subtype'])

from sklearn.model_selection import train_test_split
X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.33, random_state=42)

mlp = MLPClassifier()
mlp.fit(X_train, t_train)

y_test = mlp.predict(X_test)

/opt/tljh/user/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [49]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
import numpy as np
import sklearn

X = df[['ESR1','AURKA','ERBB2']]
t = pd.get_dummies(df['Subtype'])

pca = PCA(n_components=3)
Xnew = pd.DataFrame(pca.fit_transform(X),columns=["PC1","PC2","PC3"])

from sklearn.model_selection import train_test_split
X_train, X_test, t_train, t_test = train_test_split(Xnew, t, test_size=0.33, random_state=42)

mlp = MLPClassifier()
mlp.fit(X_train, t_train)

y_test_pca = mlp.predict(X_test)

/opt/tljh/user/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [50]:
print(sklearn.metrics.classification_report(t_test,y_test))

              precision    recall  f1-score   support

           0       0.90      0.74      0.81       112
           1       0.72      0.18      0.29        71
           2       0.65      0.66      0.66       246
           3       0.75      0.46      0.57       166
           4       0.86      0.38      0.53        50
           5       0.00      0.00      0.00        59

   micro avg       0.73      0.50      0.60       704
   macro avg       0.65      0.40      0.48       704
weighted avg       0.68      0.50      0.56       704
 samples avg       0.49      0.50      0.49       704



/opt/tljh/user/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/tljh/user/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [51]:
print(sklearn.metrics.classification_report(t_test,y_test_pca))

              precision    recall  f1-score   support

           0       0.91      0.77      0.83       112
           1       0.64      0.45      0.53        71
           2       0.71      0.70      0.70       246
           3       0.72      0.55      0.63       166
           4       0.81      0.50      0.62        50
           5       0.00      0.00      0.00        59

   micro avg       0.75      0.58      0.65       704
   macro avg       0.63      0.49      0.55       704
weighted avg       0.69      0.58      0.62       704
 samples avg       0.57      0.58      0.57       704



/opt/tljh/user/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/tljh/user/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
